In [ ]:
# !pip install -q tensorflow-model-optimization

In [1]:
%load_ext autotime

!nvidia-smi -L

import os

os.environ['CUDA_VISIBLE_DEVICES']='0'

GPU 0: NVIDIA GeForce RTX 3090 (UUID: GPU-3b49e2b8-87f0-c515-798b-3492ec05a183)
GPU 1: NVIDIA GeForce GTX 1080 Ti (UUID: GPU-07628ed7-6ef8-fd67-7d03-cb6a89f72de4)


In [2]:
import tensorflow_model_optimization as tfmot

import numpy as np, tensorflow as tf, matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras import layers
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
from tensorflow.keras.models import Model

from sklearn.metrics import confusion_matrix
import itertools, glob

# Experiment tracking with mlflow
import mlflow
import mlflow.tensorflow as mltf
from pathlib import Path
import time, multiprocessing, tarfile
from tqdm.notebook import tqdm

In [3]:
tf.config.get_visible_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
fmd = "./mlflow/artifacts/1/38162c8d183043f1bfddf866e1ee9175/artifacts/model/data/model" #final model directory

In [5]:
def get_dir_size(directory):
    root_dir=Path(".")
    size = sum(f.stat().st_size for f in root_dir.glob(directory+'/**/*') if f.is_file())
    return f"Size in MB: {size // (1024*1024)}"

In [6]:
get_dir_size(fmd)

'Size in MB: 56'

In [7]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("mosquito")
mltf.autolog()

mlflow.set_tags({"Pretrain Model": "Saved Model", 
                "Preprocessing" : "Keras VGG16 Preprocessing",
                "Pretrained Used Layers" : "first 13 Layers - Block4",
                "Framework": "tensorflow.keras"})

In [8]:
train_path = "./dataset/data_splitting/Train/"
valid_path = "./dataset/data_splitting/Test/"
test_path = "./dataset/data_splitting/Pred/"

In [9]:
# You can add more augmentations, if you want

train_gen = ImageDataGenerator(
    rotation_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=preprocess_input,
)

gen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

In [10]:
mlflow.log_params({"train ImageDataGenerator": {"rotation_range": 0.2,
                                                "horizontal_flip": True,
                                                "vertical_flip": True,
                                                "preprocessing_function": preprocess_input},
                   "test and valid ImageDataGenerator":{"preprocessing_function": preprocess_input}})

In [11]:
targetMap='''aegypti landing
aegypti smashed
albopictus landing
albopictus smashed
Culex landing
Culex smashed'''.split('\n')
targetMap

['aegypti landing',
 'aegypti smashed',
 'albopictus landing',
 'albopictus smashed',
 'Culex landing',
 'Culex smashed']

In [12]:
# Hyper-Parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 3
NUM_CLASSES = len(targetMap)

In [13]:
mlflow.log_params({"IMG_SIZE":IMG_SIZE, "Format":"RGB", "BATCH_SIZE": BATCH_SIZE, "EPOCHS": EPOCHS,
                   "NUM_CLASSES": len(targetMap)})

In [14]:
train = train_gen.flow_from_directory(train_path, target_size=IMG_SIZE,
                                      classes=targetMap, class_mode='categorical', batch_size=BATCH_SIZE)
valid = gen.flow_from_directory(valid_path, target_size=IMG_SIZE,
                                      classes=targetMap, class_mode='categorical', batch_size=BATCH_SIZE)
test = gen.flow_from_directory(test_path, target_size=IMG_SIZE,
                                      classes=targetMap, class_mode='categorical', batch_size=BATCH_SIZE)

Found 4200 images belonging to 6 classes.
Found 1799 images belonging to 6 classes.
Found 3600 images belonging to 6 classes.


In [15]:
base_model = keras.models.load_model(fmd)
base_model.layers

# But there are some problems 
1. the Rescaling layer is not supported for pruning.
2. Functional layer (the way we used vgg model) is not supported as well

#### Also there are some recommendations :
* It's generally better to finetune with pruning as opposed to training from scratch.
* Try pruning the later layers instead of the first layers.
* Avoid pruning critical layers (e.g. attention mechanism).

In [16]:
# def prune_dense(layer):
#     if isinstance(layer, tf.keras.layers.Dense):
#         return tfmot.sparsity.keras.prune_low_magnitude(layer)
#     return layer

In [17]:
def prune_custom_layer(layer):
    # prunning_params is optional (don't use it if you want)
#     end_step = np.ceil(train.n/BATCH_SIZE).astype(np.int32) * EPOCHS
#     prunning_params = {
#         tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.5, 
#                                              final_sparsity=0.9,
#                                              begin_step=0,
#                                             end_step=end_step)
#     }
    try:
        return tfmot.sparsity.keras.prune_low_magnitude(layer, **prunning_params)
    except:
        return layer

In [18]:
model = tf.keras.models.clone_model(
    base_model,
    clone_function=prune_custom_layer
)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 model (Functional)          (None, 28, 28, 512)       7635264   
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 512)      2048      
 ormalization)                                                   
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 256)       1179904   
                                                                 
 batch_normalization_1 (Batc  (None, 26, 26, 256)      1024      
 hNormalization)                                           

In [19]:
model.compile(optimizer='adam', loss=CategoricalCrossentropy(from_logits=True), metrics=['acc'])

In [20]:
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep()
]
# use tfmot.sparsity.keras.PruningSummaries(log_dir=log_dir) to log metrics on Tensorboard

h = model.fit(train, validation_data=valid, callbacks=callbacks, epochs=EPOCHS)

Epoch 1/3
132/132 [==============================] - 42s 290ms/step - loss: 0.1975 - acc: 0.9405 - val_loss: 0.4933 - val_acc: 0.8810
Epoch 2/3
132/132 [==============================] - 36s 270ms/step - loss: 0.1587 - acc: 0.9524 - val_loss: 0.2453 - val_acc: 0.9389
Epoch 3/3
132/132 [==============================] - 35s 268ms/step - loss: 0.1130 - acc: 0.9626 - val_loss: 0.1594 - val_acc: 0.9544


INFO:tensorflow:Assets written to: /tmp/tmped8v6t4w/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/tmped8v6t4w/model/data/model/assets


In [21]:
# you may see around around 2% gain in test accuracy 
# (not the case for well trained base_model)

model.evaluate(test) 

113/113 [==============================] - 6s 56ms/step - loss: 0.1163 - acc: 0.9700


[0.11631447076797485, 0.9700000286102295]

In [22]:
pfmd = f"./mlflow/artifacts/1/{mlflow.active_run().info.run_id}/artifacts/model/data/model"
get_dir_size(pfmd)

'Size in MB: 56'

# Wait, what just happened ?

__Why our pruned model is bigger than base model ?__  
* _Short answer that's not the correct way to save a prunned model_
* I see this exact problem alot in the Industry

* *Both `tfmot.sparsity.keras.strip_pruning` and applying a standard compression algorithm `(e.g. via gzip)` are necessary to see the compression benefits of pruning.*  
    * Applying a standard compression algorithm is necessary since the serialized weight matrices are the same size as they were before pruning. However, pruning makes most of the weights zeros, which is added redundancy that algorithms can utilize to further compress the model. 

* __`strip_pruning` is necessary since it removes every `tf.Variable` that pruning only needs during training, which would otherwise add to model size during inference__

In [23]:
model_to_export = tfmot.sparsity.keras.strip_pruning(model)

In [24]:
keras.models.save_model(model_to_export, "./optimized/pruned_model")

INFO:tensorflow:Assets written to: ./optimized/pruned_model/assets


INFO:tensorflow:Assets written to: ./optimized/pruned_model/assets


In [25]:
get_dir_size("./optimized/pruned_model/")

'Size in MB: 38'

In [26]:
model = keras.models.load_model("./optimized/pruned_model/", compile=False)
# For evaluation, you must run compile again (but don't need it for production as you only use predict)

In [27]:
for i in range(10):
    imgs, _ = test.next()
    for img in imgs:
        img = np.expand_dims(img, axis=0)
        model.predict(img, verbose=0)

## As recommended by TF let's use gzip

In [28]:
def gzipit(input_dir, output_file):
    root_dir=Path(".")
    with tarfile.open(output_file+".tgz", "w:gz") as tar:
        for f in root_dir.glob(input_dir+'/*'):
            tar.add(f)

In [29]:
os.makedirs("./optimized/pruned_gziped/", exist_ok=True)
gzipit("./optimized/pruned_model", "./optimized/pruned_gziped/model")
get_dir_size("./optimized/pruned_gziped/")

'Size in MB: 35'

## Now let use Dynamic Range quantization on top of pruned version

In [30]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_to_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
drq_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpiddhthaz/assets


INFO:tensorflow:Assets written to: /tmp/tmpiddhthaz/assets


In [31]:
file = "./optimized/pruned_drq/mosqueto_quant.tflite"

with open(file, 'wb') as f:
    f.write(drq_model)

os.makedirs("./optimized/pruned_gziped_tflite/", exist_ok=True)
gzipit("./optimized/pruned_drq/", "./optimized/pruned_gziped_tflite/model")

In [32]:
s1 = get_dir_size(fmd)
s2 = get_dir_size("./optimized/pruned_gziped")
s3 = get_dir_size("./optimized/pruned_gziped_tflite")

print(f"No optimization model: {s1} only quantized: 9 MB")    
print(f"gzip pruned: {s2}")
print(f"Pruned and quantized gzipped: {s3}")
fraq = int(s1.split(" ")[-1])//int(s3.split(" ")[-1])
print(f"\nMore than {fraq}x saved")

No optimization model: Size in MB: 56 only quantized: 9 MB
gzip pruned: Size in MB: 35
Pruned and quantized gzipped: Size in MB: 7

More than 8x saved
